## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=['EIN','NAME'])

In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_df["APPLICATION_TYPE"].value_counts().sort_values()

T17        1
T29        2
T15        2
T14        3
T25        3
T2        16
T12       27
T13       66
T9       156
T10      528
T7       725
T8       737
T19     1065
T5      1173
T6      1216
T4      1542
T3     27037
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=['T10',
 'T3',
 'T5',
 'T7',
 'T4',
 'T6',
 'T2',
 'T9',
 'T19',
 'T8',
 'T13',
 'T12',
 'T29',
 'T25',
 'T14',
 'T17',
 'T15']

# Replace in dataframe
for app in application_types_to_replace:
  if 0 <= application_df["APPLICATION_TYPE"].value_counts().loc[app] < 4:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Low")
  elif 16<=application_df["APPLICATION_TYPE"].value_counts().loc[app]<66:
    application_df["APPLICATION_TYPE"]=application_df["APPLICATION_TYPE"].replace(app,"Moderate")
  elif 156<=application_df["APPLICATION_TYPE"].value_counts().loc[app]<737:
    application_df["APPLICATION_TYPE"]=application_df["APPLICATION_TYPE"].replace(app,"High")
  else:
    application_df["APPLICATION_TYPE"]=application_df["APPLICATION_TYPE"].replace(app,"Very High")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

Very High    32836
High          1409
Moderate        43
Low             11
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
elem_list=application_df["CLASSIFICATION"].value_counts().index.tolist()

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for elem in elem_list:
  if 0 <= application_df["CLASSIFICATION"].value_counts().loc[elem] < 7:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(elem,"Type1")
  elif 7 <= application_df["CLASSIFICATION"].value_counts().loc[elem] < 30:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(elem,"Type2")
  elif 30 <= application_df["CLASSIFICATION"].value_counts().loc[elem] < 194:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(elem,"Type3")
  else:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(elem,"Type4")


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE


# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

Type4    33296
Type3      776
Type2      154
Type1       73
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df.info()
dummyvariables=pd.get_dummies(application_df[["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"]])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [10]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
application_df=application_df.drop(columns=["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"],axis=1)
applications=pd.concat([application_df,dummyvariables])
X=applications.drop(columns=["IS_SUCCESSFUL"])
y=applications["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=78)
X_train.shape

(51448, 36)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test.shape

(17150, 36)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
import tensorflow as tf
from tensorflow import keras
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=72, activation='relu', input_dim=36))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=72, activation='relu'))
nn.add(tf.keras.layers.Dense(units=72, activation='relu'))
nn.add(tf.keras.layers.Dense(units=72, activation='relu'))
nn.add(tf.keras.layers.Dense(units=72, activation='relu'))
nn.add(tf.keras.layers.Dense(units=72, activation='relu'))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                2664      
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)             (None, 72)                5256      
                                                                 
 dense_6 (Dense)             (None, 1)                 7

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [14]:
# Train the model
nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1608/1608 [==============================] - 12s 5ms/step - loss: nan - accuracy: 0.2332
Epoch 2/100
1608/1608 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 3/100
1608/1608 [==============================] - 4s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 4/100
1608/1608 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 5/100
1608/1608 [==============================] - 6s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 6/100
1608/1608 [==============================] - 4s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 7/100
1608/1608 [==============================] - 4s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 8/100
1608/1608 [==============================] - 6s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 9/100
1608/1608 [==============================] - 4s 3ms/step - loss: nan - accuracy: 0.2332
Epoch 10/100
1608/1608 [==============================] - 4s 3ms/step - loss: nan - accuracy: 0.233

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

536/536 - 1s - loss: nan - accuracy: 0.2357 - 962ms/epoch - 2ms/step
Loss: nan, Accuracy: 0.23568512499332428


In [18]:
# Export our model to HDF5 file
nn.save("alphabetcharitysoup_optimization.h5")